In [30]:
%%file view_abtest.py
# -*- coding: UTF-8 -*-

# author: leepand
# time: 2018-12-01
# desc: abtesting view and api
from view_base import *
from Arthur.ABtesting.utils.models import Experiment
import Arthur.ABtesting.utils.db as db
from markdown import markdown

def determine_period():
    per={'period':'day'}
    period = per.get('period', 'day')
    if period not in ['day', 'week', 'month', 'year']:
        err = {'error': 'invalid argument: {0}'.format(period), 'status': 400}
        abort(400, jsonify(err))
    return period
def simple_markdown(experiment):
    description = experiment['description']
    if description and description != '':
        experiment['pretty_description'] = markdown(description)
    return experiment
def experiment_list():
    experiments = Experiment.all(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments
def archived():
    experiments = Experiment.archived(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments

def paused():
    experiments = Experiment.paused(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]#[exp.name for exp in experiments]
    return experiments

class ViewAbtest(ViewBase):
    def GET(self):
        bRet, sRet = self.check_login()
        if not bRet:
            Log.err("user not login!")
            return web.seeother("/login")
        exp_list=experiment_list()+archived()+paused()
        if len(exp_list)>0:
            for exp_num in range(len(exp_list)):
                exp_list[exp_num]['id']=exp_num
                if exp_list[exp_num]['is_paused']:#如果实验结束，但未决胜负，则状态黑色2，否则绿色1
                    if exp_list[exp_num]['has_winner']:
                        exp_list[exp_num]['is_paused']={'result':u'已结束','status':1}
                        #exp_list[exp_num]['has_winner']={'result':u'已决','status':1}
                    else:
                        exp_list[exp_num]['is_paused']={'result':u'已结束','status':2}
                        #exp_list[exp_num]['has_winner']={'result':u'决战中','status':0}
                else:
                    exp_list[exp_num]['is_paused']={'result':u'进行中','status':0}
                if exp_list[exp_num]['has_winner']:
                    exp_list[exp_num]['has_winner']={'result':u'已决','status':1}
                else:
                    exp_list[exp_num]['has_winner']={'result':u'决战中','status':0}
                if len(exp_list[exp_num]['winner'])<1:
                    exp_list[exp_num]['winner']=u'暂无'

        return render.abtest(exp_list)

    def POST(self):
        return self.GET()
class ViewExpdetails(ViewBase):
    def GET(self,name):
        bRet, sRet = self.check_login()
        if not bRet:
            Log.err("user not login!")
            return web.seeother("/login")
        exp_list=experiment_list()+archived()+paused()
        if len(exp_list)>0:
            for exp_num in range(len(exp_list)):
                if exp_list[exp_num]['name']==name:
                    exp_out=exp_list[exp_num]['alternatives']
        
        print exp_out,'exp_outexp_outexp_out',name
        return render.expdetails(exp_out,name)

    def POST(self):
        return self.GET(name)  
    
class indexeeee:
    def GET(self):
        conn = sqlite3.connect(db.DB)
        cur = conn.cursor()
        cur.execute(db.SELECT_JOB)
        jobs = []
        for job in cur:
            state = job[0]
            state_text = setting.STATUS[int(job[0])]
            name = job[1]
            url = job[2]
            exe_time = job[3]
            id = job[4]
            dic_job = {'state': state, 'state_text': state_text, 'name': name, 'url': url,
                       'exe_time': exe_time, 'id': id}
            jobs.append(dic_job)
        return render.index(jobs)


Overwriting view_abtest.py


In [25]:
tt={'total_participants': 1, 'description': None, 'traffic_fraction': 1.0, 'is_archived': False, 'alternatives': [{'is_control': True, 'conversion_rate': 0.0, 'confidence_interval': 0, 'name': 'recom_annoy', 'participant_count': 0, 'completed_count': 0, 'is_winner': True, 'data': [{'conversions': 0.0, 'date': '2018-10-23', 'participants': 0.0}], 'test_statistic': 'N/A', 'confidence_level': 'N/A'}, {'is_control': False, 'conversion_rate': 100.0, 'confidence_interval': 0.0, 'name': 'recom_random', 'participant_count': 1, 'completed_count': 1, 'is_winner': False, 'data': [{'conversions': 1.0, 'date': '2018-10-23', 'participants': 1.0}], 'test_statistic': 'N/A', 'confidence_level': 'N/A'}], 'period': 'day', 'excluded_clients': 0, 'kpis': [], 'is_paused': False, 'name': 'recom_new1', 'created_at': '2018-10-23 12:13', 'winner': 'recom_annoy', 'has_winner': True, 'kpi': None, 'total_conversions': 1}

In [26]:
tt['alternatives']

[{'completed_count': 0,
  'confidence_interval': 0,
  'confidence_level': 'N/A',
  'conversion_rate': 0.0,
  'data': [{'conversions': 0.0, 'date': '2018-10-23', 'participants': 0.0}],
  'is_control': True,
  'is_winner': True,
  'name': 'recom_annoy',
  'participant_count': 0,
  'test_statistic': 'N/A'},
 {'completed_count': 1,
  'confidence_interval': 0.0,
  'confidence_level': 'N/A',
  'conversion_rate': 100.0,
  'data': [{'conversions': 1.0, 'date': '2018-10-23', 'participants': 1.0}],
  'is_control': False,
  'is_winner': False,
  'name': 'recom_random',
  'participant_count': 1,
  'test_statistic': 'N/A'}]

In [5]:
from Arthur.ABtesting.utils.models import Experiment
import Arthur.ABtesting.utils.db as db
from markdown import markdown

def determine_period():
    per={'period':'day'}
    period = per.get('period', 'day')
    if period not in ['day', 'week', 'month', 'year']:
        err = {'error': 'invalid argument: {0}'.format(period), 'status': 400}
        abort(400, jsonify(err))
    return period
def simple_markdown(experiment):
    description = experiment['description']
    if description and description != '':
        experiment['pretty_description'] = markdown(description)
    return experiment
def experiment_list():
    experiments = Experiment.all(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments
def archived():
    experiments = Experiment.archived(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments

def paused():
    experiments = Experiment.paused(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]#[exp.name for exp in experiments]
    return experiments

exp_list=experiment_list()+archived()+paused()
is_win_list=[]
is_pause_list=[]
is_end_list = []
if len(exp_list)>0:
    for exp_num in range(len(exp_list)):
        exp_list[exp_num]['id']=exp_num
        if exp_list[exp_num]['is_paused']:#如果实验结束，但未决胜负，则状态黑色2，否则绿色1
            is_pause_list.append(exp_list[exp_num])
        if exp_list[exp_num]['is_paused'] or exp_list[exp_num]['has_winner']:
            is_end_list.append(exp_list[exp_num])
        if exp_list[exp_num]['has_winner']:
            is_win_list.append(exp_list[exp_num])

In [4]:
len(exp_list)

20

In [7]:
len(is_win_list)

3

In [8]:
len(is_pause_list)

2

In [9]:
len(is_end_list)

4

In [10]:
is_end_list

[{'alternatives': [{'completed_count': 284,
    'confidence_interval': 0.8045201473460292,
    'confidence_level': 'N/A',
    'conversion_rate': 11.24,
    'data': [{'conversions': 284.0,
      'date': '2018-10-14',
      'participants': 2526.0}],
    'is_control': True,
    'is_winner': False,
    'name': 'default',
    'participant_count': 2526,
    'test_statistic': 'N/A'},
   {'completed_count': 338,
    'confidence_interval': 0.875036244078158,
    'confidence_level': '95% Confidence',
    'conversion_rate': 13.51,
    'data': [{'conversions': 338.0,
      'date': '2018-10-14',
      'participants': 2501.0}],
    'is_control': False,
    'is_winner': True,
    'name': 'alt1',
    'participant_count': 2501,
    'test_statistic': 5.99},
   {'completed_count': 208,
    'confidence_interval': 0.7102863957077065,
    'confidence_level': '99.9% Confidence',
    'conversion_rate': 8.36,
    'data': [{'conversions': 208.0,
      'date': '2018-10-14',
      'participants': 2488.0}],
    'i

In [19]:
"width: {data}%;".format(data=int(round((len(is_win_list)/(len(exp_list)+0.0000001)),2)*100))

'width: 15%;'

In [22]:
Experiment.validate_kpi('hello-jose')

True

In [23]:
Experiment

Arthur.ABtesting.utils.models.Experiment

In [24]:
Experiment.['kpi', '123']

AttributeError: type object 'Experiment' has no attribute 'kpi'

In [38]:

from Arthur.ABtesting.utils.models import Experiment,Client
client = Client(100, redis=db.REDIS)
exp.get_alternative(client)
exp.convert(client, None, 'bananza')

<Alternative 'kpi' (Experiment 'multi-kpi2')>

In [17]:
all_of_them = Experiment.all(redis=db.REDIS)
all_of_them

[<Experiment: layerui)>,
 <Experiment: ddddd)>,
 <Experiment: leepand)>,
 <Experiment: wwa)>,
 <Experiment: benchmark-test5)>,
 <Experiment: dew)>,
 <Experiment: nwi)>,
 <Experiment: multi-kpi2)>,
 <Experiment: multi-kpi3)>,
 <Experiment: new2222)>,
 <Experiment: test)>,
 <Experiment: ddd)>,
 <Experiment: benchmark-test2)>,
 <Experiment: mhnee)>,
 <Experiment: lyyyy)>,
 <Experiment: benchmark-test6)>,
 <Experiment: dd)>,
 <Experiment: de)>,
 <Experiment: ajax)>,
 <Experiment: nnnnn)>,
 <Experiment: leepand3)>,
 <Experiment: fres)>,
 <Experiment: exp_id)>,
 <Experiment: now)>,
 <Experiment: tag-it-test)>,
 <Experiment: thrift_recom_eng_abtesing3)>,
 <Experiment: thrift_recom_eng_abtesing2)>,
 <Experiment: beyonds)>,
 <Experiment: recom_new1)>,
 <Experiment: exp_id3)>,
 <Experiment: multi-kpi)>,
 <Experiment: nnnn)>,
 <Experiment: benchmark-test4)>,
 <Experiment: exp_id5)>,
 <Experiment: arthur)>,
 <Experiment: moeee)>,
 <Experiment: wanda_test_leepand)>,
 <Experiment: inde )>]

In [48]:
exp=Experiment.find_or_create('multi-kpi3', ['kpi', '123'],redis=db.REDIS)
client = Client(100, redis=db.REDIS)
exp.get_alternative(client)
exp.convert(client, None)

<Alternative '123' (Experiment 'multi-kpi3')>

In [41]:
exp.set_kpi('bananza')
exp.convert(client, None, 'bananza2')
exp.set_kpi('bananza2')

In [30]:
Experiment.validate_kpi('bananza')

True

In [33]:
client

In [65]:
exp=Experiment.find_or_create('benchmark-test5',['default','alt1','alt2','alt3'],redis=db.REDIS)
exp.participants_by_week()
#exp.set_kpi('dddd')

AttributeError: 'Experiment' object has no attribute 'participants_by_week'

In [ ]:
def find_or_404(experiment_name):
    try:
        #experiment_name = url=urllib.unquote(experiment_name).decode('utf8')
        exp = Experiment.find(experiment_name, db.REDIS)
        if request.args.get('kpi'):
            exp.set_kpi(request.args.get('kpi'))
        return exp
    except ValueError:
        abort(404)

In [66]:
from Arthur.common.rpc import thrifty
from Arthur.ABtesting.diversion.abtesting_thrift import Abtesting_thrift
def abtest_add(exp_id,alt,client_id):
    try:
        with thrifty.Client(Abtesting_thrift) as c:
            return c.diversion(exp_id,alt,client_id)
    except:
        print 'wrong exp create'
abtest_add('exp_id',['alt','qs'],'client_id')

lookup up: <module 'Arthur.ABtesting.diversion.abtesting_thrift.Abtesting_thrift' from '/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-0.0.8-py2.7.egg/Arthur/ABtesting/diversion/abtesting_thrift/Abtesting_thrift.pyc'>
wrong exp create


In [68]:
with thrifty.Client(Abtesting_thrift) as c:
    print c.diversion('exp_id',['alt','qs'],'client_id')

lookup up: <module 'Arthur.ABtesting.diversion.abtesting_thrift.Abtesting_thrift' from '/Users/leepand/anaconda2/lib/python2.7/site-packages/Arthur-0.0.8-py2.7.egg/Arthur/ABtesting/diversion/abtesting_thrift/Abtesting_thrift.pyc'>
Diversion(status='OK', alternative='qs', experiment='exp_id', client_id='client_id')


In [73]:
from Arthur.ABtesting.utils.models import Experiment,Client
import Arthur.ABtesting.utils.db as db

exp=Experiment.find_or_create('multi-kpi3', ['kpi', '123'],redis=db.REDIS)

exp.total_participants()
exp.update_description()

In [29]:
exp.total_conversions()

1

In [6]:
exp.get_alternative_names()

['kpi', '123']

In [9]:
exp.created_at

'2019-01-14 14:59'

In [10]:
exp.control

<Alternative 'kpi' (Experiment 'multi-kpi3')>

In [15]:
exp.objectify_by_period('day')['description']

In [13]:
exp.pretty_description()

AttributeError: 'Experiment' object has no attribute 'pretty_description'

In [26]:
all_of_them[0]

<Experiment: layerui)>

In [87]:
from markdown import markdown
from flask import abort,jsonify
def determine_period():
    period = 'day'#request.args.get('period', 'day')
    if period not in ['day', 'week', 'month', 'year']:
        err = {'error': 'invalid argument: {0}'.format(period), 'status': 400}
        abort(400, jsonify(err))
    return period

def simple_markdown(experiment):
    description = experiment['description']
    if description and description != '':
        experiment['pretty_description'] = markdown(description)
    return experiment
def find_or_404(experiment_name):
    try:
        experiment_name = experiment_name
        exp = Experiment.find(experiment_name, db.REDIS)
        #if request.args.get('kpi'):
        #   exp.set_kpi(request.args.get('kpi'))
        return exp
    except ValueError:
        return False
experiment= find_or_404('multi-kpi2')#benchmark-test3 
period = determine_period()
obj = simple_markdown(experiment.objectify_by_period(period))

In [83]:
experiment= find_or_404('my_exp')
period = determine_period()
obj = simple_markdown(experiment.objectify_by_period(period))
#obj

In [102]:
find_or_404('my_的exp')

False

In [86]:
if experiment:
    print 'ff'
obj['kpis']

ff


[]

In [94]:
experiment.set_kpi('bananza2')

In [95]:
obj = simple_markdown(experiment.objectify_by_period(period))
obj

{'alternatives': [{'completed_count': 1,
   'confidence_interval': 0.0,
   'confidence_level': 'N/A',
   'conversion_rate': 100.0,
   'data': [{'conversions': 1.0, 'date': '2019-01-14', 'participants': 1.0}],
   'is_control': True,
   'is_winner': True,
   'name': 'kpi',
   'participant_count': 1,
   'test_statistic': 'N/A'},
  {'completed_count': 0,
   'confidence_interval': 0,
   'confidence_level': 'N/A',
   'conversion_rate': 0.0,
   'data': [{'conversions': 0.0, 'date': '2019-01-14', 'participants': 0.0}],
   'is_control': False,
   'is_winner': False,
   'name': '123',
   'participant_count': 0,
   'test_statistic': 'N/A'}],
 'created_at': None,
 'description': None,
 'excluded_clients': 0,
 'has_winner': True,
 'is_archived': False,
 'is_paused': False,
 'kpi': 'bananza2',
 'kpis': ['bananza2', 'bananza'],
 'name': 'multi-kpi2',
 'period': 'day',
 'total_conversions': 1,
 'total_participants': 1,
 'traffic_fraction': 1.0,
 'winner': 'kpi'}

In [96]:
obj['kpis']

['bananza2', 'bananza']

In [35]:
experiment.created_at

'2019-01-15 13:05'

In [59]:
type(experiment.is_archived())

bool

In [36]:
experiment.update_description('jaahah')

In [42]:
obj = simple_markdown(experiment.objectify_by_period(period))
obj

{'alternatives': [{'completed_count': 0,
   'confidence_interval': 0,
   'confidence_level': 'N/A',
   'conversion_rate': 0.0,
   'data': [],
   'is_control': True,
   'is_winner': None,
   'name': 'dd',
   'participant_count': 0,
   'test_statistic': 'N/A'},
  {'completed_count': 0,
   'confidence_interval': 0,
   'confidence_level': 'N/A',
   'conversion_rate': 0.0,
   'data': [],
   'is_control': False,
   'is_winner': None,
   'name': 'ddf',
   'participant_count': 0,
   'test_statistic': 'N/A'}],
 'created_at': '2019-01-15 13:05',
 'description': u'jaahah',
 'excluded_clients': 0,
 'has_winner': False,
 'is_archived': False,
 'is_paused': False,
 'kpi': None,
 'kpis': [],
 'name': 'dd',
 'period': 'day',
 'pretty_description': u'<p>jaahah</p>',
 'total_conversions': 0,
 'total_participants': 0,
 'traffic_fraction': 1.0,
 'winner': ''}

In [44]:
experiment.created_at

'2019-01-15 13:05'

In [62]:
obj.get('has_winner')

True

In [68]:
print obj['kpis']

['bananza2', 'bananza']


In [69]:
experiment= find_or_404('benchmark-test3')
period = determine_period()
obj = simple_markdown(experiment.objectify_by_period(period))

In [70]:
obj

{'alternatives': [{'completed_count': 519,
   'confidence_interval': 0.5503477845972529,
   'confidence_level': 'N/A',
   'conversion_rate': 10.34,
   'data': [{'conversions': 519.0,
     'date': '2018-10-14',
     'participants': 5017.0}],
   'is_control': True,
   'is_winner': False,
   'name': 'default',
   'participant_count': 5017,
   'test_statistic': 'N/A'},
  {'completed_count': 599,
   'confidence_interval': 0.5934605142886294,
   'confidence_level': '99% Confidence',
   'conversion_rate': 12.1,
   'data': [{'conversions': 599.0,
     'date': '2018-10-14',
     'participants': 4949.0}],
   'is_control': False,
   'is_winner': False,
   'name': 'alt1',
   'participant_count': 4949,
   'test_statistic': 7.74},
  {'completed_count': 459,
   'confidence_interval': 0.5164051730981423,
   'confidence_level': '95% Confidence',
   'conversion_rate': 9.06,
   'data': [{'conversions': 459.0,
     'date': '2018-10-14',
     'participants': 5064.0}],
   'is_control': False,
   'is_winner'